In [40]:
import re
import textstat
import nltk
nltk.download('punkt')
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /Users/mattia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [41]:
teststring = 'schäden erbringen .  Ja , wenn das für Sie alles bekannt war , wie konnte dann ein so kluger Mann wie Ihr Kanzlerkandidat von einem guten Investitionsstandort sprechen ? Diesen Zwiespalt und Widerspruch in Ihrer Argumentation verstehe ich nicht . Die Infrastruktur muss praktisch von Grund auf überholt werden . So ist das völlig überlastete Eisenbahnnetz in seiner Ausbauqualität sogar noch weit hinter den Stand von 1939 zurückgefallen .  Frau Präsidentin , da muss ein Blähhals am Werke sein .'

## Replacing Double Full Stops by Single Full Stops

In [42]:
import copy
def replace_double_full_stops(text):
    text_clean = re.sub("\.\s*\.",".",text) # regex pattern matches two full stops separated by an arbitrary number of whitespaces
    return text_clean

replace_double_full_stops(teststring)

'schäden erbringen .  Ja , wenn das für Sie alles bekannt war , wie konnte dann ein so kluger Mann wie Ihr Kanzlerkandidat von einem guten Investitionsstandort sprechen ? Diesen Zwiespalt und Widerspruch in Ihrer Argumentation verstehe ich nicht . Die Infrastruktur muss praktisch von Grund auf überholt werden . So ist das völlig überlastete Eisenbahnnetz in seiner Ausbauqualität sogar noch weit hinter den Stand von 1939 zurückgefallen .  Frau Präsidentin , da muss ein Blähhals am Werke sein .'

## Text Length

In [43]:
def text_length(text):
    return len(text)

text_length(teststring)

496

## Average Sentence Length


In [44]:
# Some tests to check nltk's ability of splitting a German text into sentences,
# since not every full stop indicates the end of a sentence (abbreviations, ordinal numbers, ...)
# problem: does not recognize German ordinal numbers, unless they are part of a date

'''
testsen1 = "Herr Dr. Fischer, möchten Sie einen Tee?" # pass
testsen2 = "Am 7. Januar hieß es noch, das Projekt wird sofort fertig sein, heute haben wir den 20. Januar!" # pass
testsen3 = "Am 10. Tage wurde matplotlib geschaffen, und es wurde dunkel." # fail

sentences = nltk.sent_tokenize(testsen3,language='german')
print(len(sentences))
'''

def avg_sentence_length(text):
    text_length = len(text)
    sentences = nltk.sent_tokenize(text,language='german')
    n_sentences = len(sentences)
    return text_length/n_sentences

# test to compare average sentence length calculated manually vs. with nltk -> pass

'''
testsplit = ["schäden erbringen .", 
           "  Ja , wenn das für Sie alles bekannt war , wie konnte dann ein so kluger Mann wie Ihr Kanzlerkandidat von einem guten Investitionsstandort sprechen ?",
           " Diesen Zwiespalt und Widerspruch in Ihrer Argumentation verstehe ich nicht .",
           " Die Infrastruktur muss praktisch von Grund auf überholt werden .",
           " So ist das völlig überlastete Eisenbahnnetz in seiner Ausbauqualität sogar noch weit hinter den Stand von 1939 zurückgefallen .",
           "  Frau Präsidentin , da muss ein Blähhals am Werke sein ."]

testlist = [len(string) for string in testsplit]
manual_calc = sum(testlist)/len(testlist)
print(manual_calc, avg_sentence_length(teststring))
'''

avg_sentence_length(teststring)

82.66666666666667

## Number of Profanities
The profanities stem from a predefined list of unique strings, which also includes declinations.
The list has been compiled from various online sources, and mostly comprises lighter profanities,
as we would expect from the german Bundestag. However, standard severe profanities are also included.

In [46]:
profanities = ['Idiotin', 'Idioten', 'Möchtegern', 'Stümper', 'Hinterwäldlerin', 'Drecksack', 'Arschgeige', 
               'Arschloch', 'Volltrottel', 'unterbelichtet', 'Nörglerin', 'Schwachkopf', 'Faulpelz', 
               'Dreckskerl', 'Rowdy', 'hochnäsig', 'großmäulig', 'Scheiße', 'Deppen', 'verrückter', 
               'Backpfeifengesicht', 'abgefahrener', 'Lümmel', 'Ochse', 'Nörgeln', 'Depp', 'bescheuertster', 
               'Schnapsidee', 'Trottel', 'Nervensägen', 'blöde', 'Schlitzohr', 'Hanswürste', 'Zuhälter', 
               'Bauerntölpel', 'Hetzer', 'Schnauze', 'Dummköpfin', 'spinnen', 'Hetzerin', 'hochnäsige', 
               'spießig', 'Kacke', 'Ratte', 'Lackschuhpanter', 'Heuchlerin', 'dämlicher', 'beschissene', 
               'Arsch', 'Nervensäge', 'beschissen', 'Blödmann', 'Klugscheißer', 'Bastard', 'aufgeblasener', 
               'dummer', 'lahm', 'kotzen', 'altbacken', 'dümmstes', 'idiotisch', 'Schwachköpfe', 'scheiß', 
               'abgefahren', 'Dummkopf', 'dumme', 'Dummköpfe', 'Kleingeist', 'Hornochse', 'bescheuertstes', 
               'Schlange', 'Hackfresse', 'Armleuchter', 'Dreckschwein', 'hirnrissig', 'Verrückte', 'schlampig', 
               'kacke', 'Harzer', 'Pisser', 'blödes', 'Hund', 'spinnt', 'Hornochsen', 'Abschaum', 'Stinktier', 
               'Esel', 'Amateur', 'Großmaul', 'bescheuerte', 'verrückt', 'Alleswisser', 'blöd', 'Luder', 
               'schäbiger', 'Berufsrandalierer', 'Fresse', 'Stümperin', 'Zicke', 'aufgeblasene', 'Hanswurst', 
               'Sack', 'Teufel', 'Vollpfosten', 'Ziege', 'Galgenkandidat', 'Hurensohn', 'kindisch', 'Idiot', 
               'Dreckschweine', 'schmierig', 'Verrückter', 'Angeberin', 'Schwein', 'Hinterwäldler', 'verdammte', 
               'kleingeistig', 'aufgeblasen', 'Affe', 'bescheuertste', 'versifft', 'Bastarde', 'bieder', 
               'schäbig', 'Blöde', 'Schweine', 'Gangster', 'blödsinnig', 'dumm', 'stümperhaft', 'Arschlöcher', 
               'Spießer', 'verdammt', 'bescheuert', 'affig', 'Faulpelze', 'Angeber', 'Arschgeigen', 'Aasgeier', 
               'Besserwisser', 'Blöder', 'verrückte', 'Blödmänner', 'Heuchler', 'Nörgler', 'dämlich', 'dümmste', 
               'Hurensöhne', 'heuchlerisch', 'Pisse', 'bescheuerter', 'kleinkariert', 'Karnickel', 'kleinkarierte',
               'Blähhals', 'verdammter', 'verdammtes']

def count_profanities(text):
    n_profanities = 0
    tokens = nltk.word_tokenize(text, language='german')
    for profanity in profanities:
        n_profanities += tokens.count(profanity)
    return n_profanities

count_profanities(teststring)

1

## Type-Token-Ratio (#unique words / #total words)

In [47]:
def TTR(text):
    tokens = nltk.word_tokenize(text, language='german')
    tokens = [token.lower() for token in tokens if token.isalpha()]
    n_total = len(tokens)
    n_unique = len(set(tokens))
    return n_unique/n_total

TTR("Morgen morgen")

0.5

## Readability Score
Readability is calculated as Flesch-Reading-Ease for the German language.
Interpretation: score of 0-30: very difficult, 30-50: difficult,
50-60: medium/difficult, 60-70: medium, 70-80: medium/easy, 80-90: easy,
90-100: very easy. See https://de.wikipedia.org/wiki/Lesbarkeitsindex#Flesch-Reading-Ease

In [48]:
def readability(text):
    textstat.set_lang("de")
    return textstat.flesch_reading_ease(text)

readability(teststring)

45.2

## Sentiment Analysis
<p> Based on the data set "SentiWS" (v.2.0), by R. Remus, U. Quasthoff & G. Heyer: SentiWS - a Publicly Available German-language Resource for Sentiment Analysis.
In: Proceedings of the 7th International Language Resources and Evaluation (LREC'10), pp. 1168-1171, 2010.
The data set is kindly provided by the University of Leipzig.
</p>
<p>
Source: https://wortschatz.uni-leipzig.de/de/download
</p>
<p>
The data set includes sentiment scores (range: -1 to +1) of many german words and their respective declinations,
which are assigned the same sentiment score. We compute the sentiment score of a text as the average sentiment
score of all words in that text, which have a sentiment score in the data set.
</p>

In [49]:
d = {} # create dictionary 'd' with the words as keys and their sentiment-scores as values

with open("./SentiWS_v2.0/SentiWS_v2.0_Negative.txt") as f: # read the file containing negative-sentiment words
    for line in f:
        split = re.split('\||\s|,',line) # split line at the delimiters '|', whitespace, and ','
        keys = [split[0]] + split[3:-1] # make a list containing the the word and its declinations
        value = float(split[2]) # sentiment score for the word and its declinations
        for key in keys:
            d[key] = value # add key:value pair to d for the word and its declinations

# do exaclty the same for positive-sentiment words
with open("./SentiWS_v2.0/SentiWS_v2.0_Positive.txt") as g:
    for line in g:
        split = re.split('\||\s|,',line)
        keys = [split[0]] + split[3:-1]
        value = float(split[2])
        for key in keys:
            d[key] = value

            
# print(d['ausgezeichnet']) # conflict: the word is in the data base as adjective and as past-tense verb, with diffenrent sentiment scores
            
def sentiment(text):
    tokens = nltk.word_tokenize(text, language='german')
    tokens = [token.lower() for token in tokens if token.isalpha()]
    sentiment_sum = 0 # sum up sentiment scores for tokens which have a sentiment score
    sentiment_n = 0 # count the tokens which have a sentiment score
    for token in tokens:
        sentiment_score = d.get(token)
        if sentiment_score != None:
            sentiment_sum += sentiment_score
            sentiment_n += 1
    if sentiment_n > 0:
        return sentiment_sum/sentiment_n # must be between -1 and 1
    else:
        return 0
    
sentitest_1 = "Vor ihm drängten sich und verschwanden leichthin die Häuser der Rue de Rome. Zur Linken öffneten die bedeckten Hallen ihre angerauchten Riesenglasdächer, das Auge tauchte tief in die ungeheure Halle für den Fernverkehr, welche die Baulichkeiten für die Post und das Wärmrohrmagazin von den anderen kleineren Hallen für den Verkehr nach Argenteuil, Versailles und der Ringbahn trennten. Rechts dagegen überwölbte der Pont de l'Europe mit seinem eisernen Stern die tiefe Furche, die man jenseits wieder erscheinen und von dort bis zum Tunnel von Les Batignolles heranreichen sah. Gerade unter dem Fenster, welches dieses ganze, mächtige Feld beherrschte, theilten sich die drei doppelten Schienenstränge, die unter der Brücke hervorkamen, in zahlreiche andere, die fächerartig auseinander liefen, und ihre vervielfachten, zahllosen, metallenen Arme verloren sich sofort unter den Glasdächern der Hallen. Die drei Weichenstellerhäuschen diesseits der Brückenbogen zeigten ihre öden Gärtchen. Mitten in dem konfusen Gewirr der auf den Schienen umherstehenden Waggons und Maschinen schimmerte ein rothes Signallicht verletzend durch den bleichen Tag."
sentitest_2 = "Warin war ein Graf zu Altorf und Ravensburg in Schwaben, sein Sohn hieß Isenbart und Irmentrut dessen Gemahlin. Es geschah, daß ein armes Weib unweit Altorf drei Kindlein auf ein Mal zur Welt brachte; als das Irmentrut die Gräfin hörte, rief sie aus: »Es ist unmöglich, daß dies Weib drei Kinder von einem Mann haben könne, ohne Ehbruch.« Dieses redete sie öffentlich vor Graf Isenbart ihrem Herrn und allem Hofgesinde »und diese Ehbrecherin verdiene nichts anders, als in einen Sack gesteckt und ertränkt zu werden."
sentitest_3 = "Lasst uns froh und munter sein! Und uns recht von Herzen freuen! Lustig, lustig, tralera-lera, Bald ist Nikolausabend da, Bald ist Nikolausabend da!"
    
print(sentiment(sentitest_1))

-0.21034000000000003
